In [1]:
from glob import glob
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import xgcm
from xorca.lib import load_xorca_dataset
import pickle
import eddytools as et
from cmocean import cm

import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
# Obviously it is not a great idea to ignore warnings, however there are quite many
# RuntimeWarnings because of division by 0 in some parts of this notebook. To keep
# the instructive nature of this example notebook, these warnings are ignored.

Uncomment the following cell to use `dask distributed` (and `dask.bags` in the detection).  
__This is experimental not thouroughly tested__

In [2]:
#from dask.distributed import Client, LocalCluster
#cluster = LocalCluster(n_workers=3, threads_per_worker=2)
#client = Client(cluster)
#client

In [3]:
# paths
datapath = '/home/jan/Data/ORCA_test_data/'
meshpath = [datapath + 'test_ORCA_mesh_mask.nc']
data_in = sorted(glob(datapath + 'test_data_year[12]_grid_[TUV].nc'))

### Loading data
Loading data and later calculations are much quicker when:  
1) The chunks on disk are the same size as the chunks specified in `load_xorca_dataset(..., input_ds_chunks, target_ds_chunks)`  
2) Data on disk is not compressed  
The files loaded in this notebook have been created with `ncks -7 -L 0 --cnk_dmn time_counter,73 --cnk_dmn deptht,46 --cnk_dmn y,100 --cnk_dmn x,100 ...` (for grid_T) etc.  
3) If possible, use more than one cpu. (`xarray` will automatically invoke `dask` and thus at least the calculation of the Okubo-Weiss parameter, the spatial standard deviation thereof, and the interpolation benefit from the parallelization capabilities of `dask`.)

In [4]:
# load data
data = load_xorca_dataset(data_files=data_in, aux_files=meshpath, model_config='NEST',
           input_ds_chunks = {"time_counter": 1, "t": 1,
                              "z": 11, "deptht": 11, "depthu": 11, "depthv": 11, "depthw": 11,
                              "x": 1002, "y": 629},
           target_ds_chunks = {"t": 1,
                               "z_c": 11, "z_l": 11,
                               "x_c": 1002, "x_r": 1002, "y_c": 629, "y_r": 629})

In [5]:
# define metrics for xgcm (not strictly necessary)
at, au = data['e1t'] * data['e2t'], data['e1u'] * data['e2u']
av, af = data['e1v'] * data['e2v'], data['e1f'] * data['e2f']
vt, vu, vv, vw = data['e3t'] * at, data['e3u'] * au, data['e3v'] * av, data['e3w'] * at

data = data.update({'at': at, 'au': au, 'av': av, 'af': af, 'vt': vt, 'vu': vu, 'vv': vv, 'vw': vw})
data = data.set_coords(['at', 'au', 'av', 'af', 'vt', 'vu', 'vv', 'vw'])

metrics = {
    ('X',): ['e1t', 'e1u', 'e1v', 'e1f'], # X distances
    ('Y',): ['e2t', 'e2u', 'e2v', 'e2f'], # Y distances
    ('Z',): ['e3t', 'e3u', 'e3v', 'e3w'], # Z distances
    ('X', 'Y'): ['at', 'au', 'av', 'af'], # Areas
    ('X', 'Y', 'Z'): ['vt', 'vu', 'vv', 'vw'] # Volumes
}

In [6]:
# add bathymetry to data to have depth information
bathy = xr.open_dataset(datapath + 'test_ORCA_bathy_meter.nc')
data = data.update({'bathymetry': (['y_c', 'x_c'], bathy['Bathymetry'].data)})

In [7]:
grid = xgcm.Grid(data, metrics=metrics)

In [8]:
data

<xarray.Dataset>
Dimensions:     (t: 146, x_c: 1002, x_r: 1002, y_c: 629, y_r: 629, z_c: 11, z_l: 11)
Coordinates: (12/41)
  * z_c         (z_c) int64 1 2 3 4 5 6 7 8 9 10 11
  * z_l         (z_l) float64 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5 10.5
  * y_c         (y_c) int64 1 2 3 4 5 6 7 8 ... 622 623 624 625 626 627 628 629
  * y_r         (y_r) float64 1.5 2.5 3.5 4.5 5.5 ... 626.5 627.5 628.5 629.5
  * x_c         (x_c) int64 1 2 3 4 5 6 7 8 ... 996 997 998 999 1000 1001 1002
  * x_r         (x_r) float64 1.5 2.5 3.5 4.5 ... 1e+03 1.002e+03 1.002e+03
    ...          ...
    av          (y_r, x_c) float64 dask.array<chunksize=(629, 1002), meta=np.ndarray>
    af          (y_r, x_r) float64 dask.array<chunksize=(629, 1002), meta=np.ndarray>
    vt          (z_c, y_c, x_c) float64 dask.array<chunksize=(11, 629, 1002), meta=np.ndarray>
    vu          (z_c, y_c, x_r) float64 dask.array<chunksize=(11, 629, 1002), meta=np.ndarray>
    vv          (z_c, y_r, x_c) float64 dask.array<chunksize=(11, 629, 1002), meta=np.ndarray>
    vw          (z_l, y_c, x_c) float64 dask.array<chunksize=(11, 629, 1002), meta=np.ndarray>
Data variables:
    sossheig    (t, y_c, x_c) float32 dask.array<chunksize=(1, 629, 1002), meta=np.ndarray>
    vosaline    (t, z_c, y_c, x_c) float32 dask.array<chunksize=(1, 11, 629, 1002), meta=np.ndarray>
    votemper    (t, z_c, y_c, x_c) float32 dask.array<chunksize=(1, 11, 629, 1002), meta=np.ndarray>
    vomecrty    (t, z_c, y_r, x_c) float32 dask.array<chunksize=(1, 11, 629, 1002), meta=np.ndarray>
    vozocrtx    (t, z_c, y_c, x_r) float32 dask.array<chunksize=(1, 11, 629, 1002), meta=np.ndarray>
    bathymetry  (y_c, x_c) float32 0.0 0.0 0.0 ... 5.72e+03 5.72e+03 5.72e+03

### OKUBO-WEISS
Calculation of the Okubo-Weiss paramter is only done on one depth level, the depth at which we want to detect eddies. In this case we choose `z_c=9, z_l=9` which is ~93 m depth.  
We use xgcm to define the grid for the interpolation and differentiation necessary for the calculation of vorticity and the Okubo-Weiss parameter.  
For more information on the Okubo-Weiss parameter and its calculation see [Okubo, 1970](https://doi.org/10.1016/0011-7471(70)90059-8); [Weiss, 1991](https://doi.org/10.1016/0167-2789(91)90088-Q); [Basdevant and Philipovitch, 1994](https://doi.org/10.1016/0167-2789(94)90222-4); [Chelton et al., 2007](https://doi.org/10.1029/2007GL030812).

In [9]:
# Calculate vorticity and Okubo-Weiss parameter and make sure the chunk sizes are as before.
data_OW = et.okuboweiss.calc(data.isel(z_c=9, z_l=9), grid,
                             'vozocrtx', 'vomecrty').chunk({'x_c': 1002, 'x_r': 1002,
                                                            'y_c': 629, 'y_r': 629})

In [10]:
# Merge the new variables `OW` and `vort` to the dataset `data`
data = xr.merge([data, data_OW], compat='override')

In [11]:
data.t

<xarray.DataArray 't' (t: 146)>
array(['1976-01-03T12:00:00.000000000', '1976-01-08T12:00:00.000000000',
       '1976-01-13T12:00:00.000000000', '1976-01-18T12:00:00.000000000',
       '1976-01-23T12:00:00.000000000', '1976-01-28T12:00:00.000000000',
       '1976-02-02T12:00:00.000000000', '1976-02-07T12:00:00.000000000',
       '1976-02-12T12:00:00.000000000', '1976-02-17T12:00:00.000000000',
       '1976-02-22T12:00:00.000000000', '1976-02-28T00:00:00.000000000',
       '1976-03-04T12:00:00.000000000', '1976-03-09T12:00:00.000000000',
       '1976-03-14T12:00:00.000000000', '1976-03-19T12:00:00.000000000',
       '1976-03-24T12:00:00.000000000', '1976-03-29T12:00:00.000000000',
       '1976-04-03T12:00:00.000000000', '1976-04-08T12:00:00.000000000',
       '1976-04-13T12:00:00.000000000', '1976-04-18T12:00:00.000000000',
       '1976-04-23T12:00:00.000000000', '1976-04-28T12:00:00.000000000',
       '1976-05-03T12:00:00.000000000', '1976-05-08T12:00:00.000000000',
       '1976-05-13T12:00:00.000000000', '1976-05-18T12:00:00.000000000',
       '1976-05-23T12:00:00.000000000', '1976-05-28T12:00:00.000000000',
       '1976-06-02T12:00:00.000000000', '1976-06-07T12:00:00.000000000',
       '1976-06-12T12:00:00.000000000', '1976-06-17T12:00:00.000000000',
       '1976-06-22T12:00:00.000000000', '1976-06-27T12:00:00.000000000',
       '1976-07-02T12:00:00.000000000', '1976-07-07T12:00:00.000000000',
       '1976-07-12T12:00:00.000000000', '1976-07-17T12:00:00.000000000',
       '1976-07-22T12:00:00.000000000', '1976-07-27T12:00:00.000000000',
       '1976-08-01T12:00:00.000000000', '1976-08-06T12:00:00.000000000',
       '1976-08-11T12:00:00.000000000', '1976-08-16T12:00:00.000000000',
       '1976-08-21T12:00:00.000000000', '1976-08-26T12:00:00.000000000',
       '1976-08-31T12:00:00.000000000', '1976-09-05T12:00:00.000000000',
       '1976-09-10T12:00:00.000000000', '1976-09-15T12:00:00.000000000',
       '1976-09-20T12:00:00.000000000', '1976-09-25T12:00:00.000000000',
       '1976-09-30T12:00:00.000000000', '1976-10-05T12:00:00.000000000',
       '1976-10-10T12:00:00.000000000', '1976-10-15T12:00:00.000000000',
       '1976-10-20T12:00:00.000000000', '1976-10-25T12:00:00.000000000',
       '1976-10-30T12:00:00.000000000', '1976-11-04T12:00:00.000000000',
       '1976-11-09T12:00:00.000000000', '1976-11-14T12:00:00.000000000',
       '1976-11-19T12:00:00.000000000', '1976-11-24T12:00:00.000000000',
       '1976-11-29T12:00:00.000000000', '1976-12-04T12:00:00.000000000',
       '1976-12-09T12:00:00.000000000', '1976-12-14T12:00:00.000000000',
       '1976-12-19T12:00:00.000000000', '1976-12-24T12:00:00.000000000',
       '1976-12-29T12:00:00.000000000', '1977-01-03T12:00:00.000000000',
       '1977-01-08T12:00:00.000000000', '1977-01-13T12:00:00.000000000',
       '1977-01-18T12:00:00.000000000', '1977-01-23T12:00:00.000000000',
       '1977-01-28T12:00:00.000000000', '1977-02-02T12:00:00.000000000',
       '1977-02-07T12:00:00.000000000', '1977-02-12T12:00:00.000000000',
       '1977-02-17T12:00:00.000000000', '1977-02-22T12:00:00.000000000',
       '1977-02-27T12:00:00.000000000', '1977-03-04T12:00:00.000000000',
       '1977-03-09T12:00:00.000000000', '1977-03-14T12:00:00.000000000',
       '1977-03-19T12:00:00.000000000', '1977-03-24T12:00:00.000000000',
       '1977-03-29T12:00:00.000000000', '1977-04-03T12:00:00.000000000',
       '1977-04-08T12:00:00.000000000', '1977-04-13T12:00:00.000000000',
       '1977-04-18T12:00:00.000000000', '1977-04-23T12:00:00.000000000',
       '1977-04-28T12:00:00.000000000', '1977-05-03T12:00:00.000000000',
       '1977-05-08T12:00:00.000000000', '1977-05-13T12:00:00.000000000',
       '1977-05-18T12:00:00.000000000', '1977-05-23T12:00:00.000000000',
       '1977-05-28T12:00:00.000000000', '1977-06-02T12:00:00.000000000',
       '1977-06-07T12:00:00.000000000', '1977-06-12T12:00:00.000000000',
       '1977-06-17T12:00:00.000000000', '1977-06-22T12:00:00.000000000',
       '1977-06-27T12:00:0

### INTERPOLATION
Now we interpolate the data. If the data is entirely on a grid, that has one-dimensional latitude and longitude dimensions (i.e. longitude at one x-index does not change with changing y-index and latitude at one y-index does not change with changing x-index), the fields are simply interpolated to the F-point of the grid (vorticity and OW paramter are calculated and stored already on the F-point).
If the grid is curivlinear, e.g. North of 20S or so on the ORCA grids, a regridding is performed.  
Note that the interpolation can take VERY long for large region and/or long time periods. If one has to deal with problems regarding the time it takes, one could split the interpolation into several parts, let's say interpolate one year at a time, store the results and later load the combined interpolated data.

The longitude in all the parameter definitions (`lon1` and `lon2`) needs to be in the range (-180, 180). Longitudes larger then 180 will result in an error.

IMPORTANT:  
The region that gets interpolated needs to be larger than the region used later for detection and tracking. This is required to ensure that eddies close to the boundaries of the detection region can be detected, tracked and sampled, even if their surroundings extend beyond the limits of the detection region.  
To handle this, the region is internally extended by 2 degrees in each longitude direction and 1 degree in latitude direction. So you need to make sure that the region you specify is not closer to the limits of your data than that.  
If you want to detect eddies in the region 140-170E and 50-60S, the interpolation will be carried out within 138-172E and 49-61S! However, you still need to define 140-170E and 50-60S in the `interpolation_parameters`.

If you are not 100% sure yet how big the region will be in which you want to detect the eddies, you can specify a larger region for the interpolation and then later define smaller regions for the detection, tracking, etc.

In [12]:
# Define the parameters for the interpolation
interpolation_parameters = {'model': 'ORCA',
                            'grid': 'latlon',
                            'start_time': '1976-01-01', # time range start
                            'end_time': '1976-01-31', # time range end
                            'calendar': 'standard', # calendar, must be either 360_day or standard
                            'lon1': 172, # minimum longitude of detection region
                            'lon2': -178, # maximum longitude
                            'lat1': -58, # minimum latitude
                            'lat2': -52, # maximum latitude
                            'res': 1./10., # resolution of the fields in degrees
                            'vars_to_interpolate': ['OW', 'vort', 'votemper', 'vosaline'], # variables to be interpolated 
                            'mask_to_interpolate': ['fmask', 'tmask', 'bathymetry']} # masks to interpolate

In [13]:
data_int, regridder = et.interp.horizontal(data, metrics, interpolation_parameters, weights=None)

# The variable `regridder` returns the weights if a regridding with xESMF has been performed. 
# If yes, execute the following line to save this regridder to disk, so it can be reused
#regridder.to_netcdf("test_regridder.nc")

# Then the weights can be reused for future interpolation with the sam grid (i.e. the `lon1`, `lon2`, `lat1`, `lat2` 
# in `interpolation_paramters` must be identical)
#weights = xr.open_dataset("test_regridder.nc")
#data_int, regridder = et.interp.horizontal(data, metrics, interpolation_parameters, weights=weights)

Interpolating from model grid: ORCA
No regridding necessary, just interpolating to vorticity grid point.
Interpolating OW
Interpolating vort
Interpolating votemper
Interpolating vosaline
Interpolating fmask
Interpolating tmask
Interpolating bathymetry
Interpolating e1f
Interpolating e2f


Don't be confused that interpolation of `e1f` and `e2f` is done although we did not request that. `fmask`, `e1f` and `e2f` are always interpolated, even if you do not request that because they are essential for the detection of the eddies!

### Threshold for the Okubo-Weiss parameter
Some sort of threshold needs to be defined for the Okubo-Weiss parameter to distinguish regions within and outside of eddies. This is usually defined as `OW_thr = a * sigma` ([Chelton et al., 2007](https://doi.org/10.1029/2007GL030812)), where `sigma` is the spatial standard deviation of the Okubo-Weiss parameter and `a` is some factor (more on the factor later).  
In regions with highly variable dynamic conditions, detection based on a single values of this standard deviation often fails to detect eddies in more "quiet" areas of the ocean ([Isern-Fontanet et al., 2003](https://doi.org/10.1175/1520-0426(2003)20<772:IOMEFA>2.0.CO;2)).Thus we here define a spatially variable threshold for the Okubo-Weiss parameter by calculating the (2D) spatial standard deviation in moving boxes.

In [14]:
# Load `OW` into memory so the `.rolling` operation is faster
OW_tmp = data_int['OW'].compute()

In [15]:
 # Convert all land values to `NaN` so we don't have a lot of zeros when calculating
# the standard deviation
OW_tmp = OW_tmp.where(OW_tmp != 0)
lon_tmp = OW_tmp['lon'].where(OW_tmp['lon'] > 0, other=OW_tmp['lon'] + 360.)
OW_tmp = OW_tmp.assign_coords({'lon': lon_tmp})

In [16]:
mean_OW_spatial_std = OW_tmp.rolling(
                          lon=100, center=True, min_periods=1
                          ).std(skipna=True).rolling(
                          lat=100, center=True, min_periods=1
                          ).std(skipna=True).mean('time')

It is also possible to use the mean spatial standard deviation of the Okubo-Weiss parameter over the whole considered region. To do that execute the cell below instead of the one above! Please note that `'OW_thr'` in `detection_parameters` needs to be defined differently, depending on which method of calculating the standard deviation you choose.

In [17]:
#mean_OW_spatial_std = OW_tmp.std(('lon', 'lat'), skipna=True).mean('time') # uncomment if necessary

In [18]:
# use if OW_std is 2D
data_int = data_int.update({'OW_std': (['lat', 'lon'], mean_OW_spatial_std.values)})

In [19]:
# use if OW_std is 1D
#data_int = data_int.update({'OW_std': mean_OW_spatial_std})

In [20]:
 # Store the interpolated field on disk so we do not have to do the interpolation again
data_int.to_netcdf(datapath + 'test_data_OW.nc')

In [ ]:
# If we already did the interpolation earlier and now start off with the interpolated fields,
# we just uncomment the line below to load them
data_int = xr.open_dataset(datapath + 'test_data_OW.nc')

In [ ]:
# Let's have a look at the fields
plt.pcolormesh(data_int['OW'].isel(time=0), vmin=-5e-10, vmax=5e-10, cmap=cm.balance)

We see several eddy-like features that have a low OW-parameter, now let's see whether the algorithm detects them as eddies!

### DETECTION

Things to note for the setting of `detection_parameters`:  
1. `'lon1'` and `'lon2'` need to be in the range (-180, 180). `'lon2'` can be smaller than `'lon1'` if you want to extract work in a region extending across the dateline (e.g. `'lon1': 160`, `'lon2': -170`). 
2. `'OW_thr': data_int` if the field `OW_std` in `data_int` is 2D, i.e. we have a spatially varying standard deviation of the Okubo-Weiss parameter. `'OW_thr': data_int['OW_std'].values` if the field `OW_std` in `data_int` is 1D.  
3. `'OW_thr_factor'` is usually chosen in the range `(-0.5, -0.2)` ([Chelton et al., 2007](https://doi.org/10.1029/2007GL030812); [Isern-Fontanet et al., 2003](https://doi.org/10.1175/1520-0426(2003)20<772:IOMEFA>2.0.CO;2)).

In [ ]:
# Specify parameters for eddy detection
detection_parameters = {'model': 'ORCA',
                        'grid': 'latlon',
                        'start_time': '1976-01-01', # time range start
                        'end_time': '1976-12-31', # time range end
                        'calendar': 'standard', # calendar, must be either 360_day or standard
                        'lon1': 172, # minimum longitude of detection region
                        'lon2': -178, # maximum longitude
                        'lat1': -58, # minimum latitude
                        'lat2': -52, # maximum latitude
                        'min_dep': 1000, # minimum ocean depth where to look for eddies in m
                        'res': 1./10., # resolution of the fields in degree
                        'OW_thr': data_int['OW_std'].values, # 
                        'OW_thr_name': 'OW_std', # Okubo-Weiss threshold for eddy detection
                        'OW_thr_factor': -0.3, # Okubo-Weiss parameter threshold
                        'Npix_min': 20, # minimum number of pixels (grid cells) to be considered as eddy
                        'Npix_max': 500} # maximum number of pixels (grid cells)

In [ ]:
# We detect eddies at level 10 [.isel(z=9)] (as we only calculated the Okubow-Weiss
# parameter at this level)
eddies = et.detection.detect_OW(data_int.isel(z=9), detection_parameters, 'OW', 'vort')

In [ ]:
# In this format, information for every detected eddy is stored
# (eddies[t][ed], where t is the time step and ed the eddy number)
eddies[0][0]

In [ ]:
# Plot a single time step to see what the detection method detected
t=0
d_p = detection_parameters
plot_lon = data_int['lon'].where(data_int['lon'].values > 0, other=data_int['lon'].values + 360)
plt.pcolormesh(plot_lon, data_int['lat'].values, data_int.OW.isel(time=t).values,
               vmin=-1e-9, vmax=1e-9, cmap=cm.balance, shading='auto')
for c, l in zip(['whitesmoke', 'dimgray'], ['-', '--']):
    plt.plot([d_p['lon1'], d_p['lon2'], d_p['lon2'], d_p['lon1'], d_p['lon1']],
             [d_p['lat1'], d_p['lat1'], d_p['lat2'], d_p['lat2'], d_p['lat1']],
             color=c, linestyle=l)

for i in np.arange(0, len(eddies[t])):
    if eddies[t][i]['lon'] < 0:
        eddy_lon = eddies[t][i]['lon'] + 360
    else:
        eddy_lon = eddies[t][i]['lon']
    if eddies[t][i]['type'] == 'cyclonic':
        for s, c in zip([7, 6], ['k', 'gold']):
            plt.plot(eddy_lon, eddies[t][i]['lat'], marker='o', color=c, markersize=s)
    elif eddies[t][i]['type'] == 'anticyclonic':
        for s, c in zip([7, 6], ['k', 'silver']):
            plt.plot(eddy_lon, eddies[t][i]['lat'], marker='o', color=c, markersize=s)

We see that all features below a certain OW threshold have been detected. Some of them, we might not consider as an eddy. If you have the feeling, that too many features are detected, that are not eddies, change parameters like `OW_thr_factor`, `Npix_min`, `Npix_max`

In [ ]:
# For every time step, we store one file on disk with all the information of the detected
# eddies at this time step
for i in np.arange(0, len(eddies)):
    datestring = str(eddies[i][0]['time'])[0:10]
    with open(datapath + 'test_'
          + str(datestring) + '_eddies_OW0.3_test.pickle', 'wb') as f:
        pickle.dump(eddies[i], f, pickle.HIGHEST_PROTOCOL)
    f.close()

### TRACKING

Things to note for the setting of `tracking_parameters`:  
1. `'start_time'` is required to be no earlier than the earliest actual date of the detected eddies. In our case here, for the year 2018 and a 5-day temporal resolution of the data, this is `'2018-01-03'` (if we had daily data, it would be `'2018-01-01'`).
2. `'lon1'` and `'lon2'` need to be identical to `'lon1'` and `'lon2'` in `detection_parameters`.  
3. If you stored the detected eddies in files and want to track these, set `'dict': 0` and make sure `'data_path'`, `'file_root'` and `'file_spec'` are set accordingly. The method will look for files `datapath + file_root + 'YYYYMMDD' + file_spec + '.pickle'`, the date `'YYYYMMDD'` is automatically calculated from `'start_time'`, `'dt'`, and `'end_time'`. You have to make sure that the stored, detected eddies contain that date in their filename (e.g. as defined in the cell above)!

In [ ]:
# Specify parameters for eddy tracking
tracking_parameters = {'model': 'ORCA',
                       'grid': 'latlon',
                       'start_time': '1976-01-03', # time range start
                       'end_time': '1976-12-31', # time range end
                       'calendar': 'standard', # calendar, must be either 360_day or standard
                       'dt': 5, # temporal resolution of the data in days
                       'lon1': 172, # minimum longitude of detection region
                       'lon2': -178, # maximum longitude
                       'lat1': -58, # minimum latitude
                       'lat2': -52, # maximum latitude
                       'dE': 50., # maximum distance of search ellipsis from eddy center in towards the east 
                                 # (if set to 0, it will be calculated as (150. / (7. / dt)))
                       'eddy_scale_min': 0.5, # minimum factor by which eddy amplitude and area are allowed to change in one timestep
                       'eddy_scale_max': 1.5, # maximum factor by which eddy amplitude and area are allowed to change in one timestep
                       'dict': eddies, # dictionary containing detected eddies to be used when not stored in files (set to 0 otherwise)
                       'data_path': datapath, # path to the detected eddies pickle files
                       'file_root': 'test',
                       'file_spec': 'eddies_OW0.3_test',
                       'ross_path': datapath} # path to rossrad.dat containing Chelton et a1. 1998 Rossby radii

In [ ]:
# Now we track the eddies, all information needed has to be added to `tracking_parameters`
tracks = et.tracking.track(tracking_parameters, in_file=False)

We now have tracked all eddies that met the criteria specified in `tracking_parameters`. Every entry `i` in `tracks[i]` corresponds to one complete track.

In [ ]:
# The entries in `track` look like this
tracks[1]

In [ ]:
# To have a look at how the tracking performs, just pick two eddies and see whether they are tracked.
ed1 = 0
ed2 = 4
t = 0
j = 5

plt.figure(figsize=(18, j))

plot_lon = data_int['lon'].where(data_int['lon'].values > 0, other=data_int['lon'].values + 360)

ed1_lon = tracks[ed1]['lon']
ed1_lon[ed1_lon < 0] = ed1_lon[ed1_lon < 0] + 360
ed2_lon = tracks[ed2]['lon']
ed2_lon[ed2_lon < 0] = ed2_lon[ed2_lon < 0] + 360

for i in np.arange(0, j):
    plt.subplot(1, j, i + 1)
    plt.pcolormesh(plot_lon, data_int.lat, data_int.OW.isel(time=t + i).values,
                   vmin=-5e-10, vmax=5e-10, cmap=cm.balance, shading='auto')
    plt.plot(ed1_lon[t:t + i+1], tracks[ed1]['lat'][t:t + i+1], marker='o', color='m')
    plt.plot(ed2_lon[t:t + i+1], tracks[ed2]['lat'][t:t + i+1], marker='o', color='y')

In [ ]:
 # We save the tracks for later use
with open(datapath
          + 'test_19760101_19761231_tracks_OW0.3'
          + '_test.pickle', 'wb') as f:
    pickle.dump(tracks, f, pickle.HIGHEST_PROTOCOL)
f.close()

In [ ]:
# This is how to open the tracks-file again (no need to do that if we just saved it)
with open(datapath
          + 'test_19760101_19761231_tracks_OW0.3'
          + '_test.pickle', 'rb') as f:
    tracks = pickle.load(f)
f.close()

### SAMPLING

Things to note for the setting of `sample_parameters`:  
1. `'start_time'` is required to be no earlier than the earliest actual date of the tracked eddies. In our case here, for the year 2018 and a 5-day temporal resolution of the data, this is `'2018-01-03'` (if we had daily data, it would be `'2018-01-01'`).
2. `'lon1'` and `'lon2'` need to be identical to `'lon1'` and `'lon2'` in `detection_parameters`.  
3. Right now, the usage of `'range'` and `'split'` has not been thouroughly tested! It seems to work for most cases though.  

`'range'`: Set to `True` if you only want to sample eddies within a certain range `'values_range'` of a property `'var_range'` in the dataset `'ds_range'`. `'var_range'` needs to be 2D (thus the `.isel(z=9)` in the example below) and interpolated to the same grid as `OW` used above. It is most likely that, if you follow this example, `'var_range'` is stored in the same dataset as `OW`. In the example below, only eddies that have a center temperature between 4 and 7 degrees C at depth level 10 (`z=9`) will be sampled and stored.  

`'split'`: Set to `True` if you want to split the sampled eddies into two categories, above and below a certain threshold value `'value_split'` of a variable `'var_split'` in the dataset `'ds_split'`. As for `'range'`, `'var_split'` needs to be 2D and interpolated to the same grid as `OW` used above. In the example below the eddies will be put into two categories: In the first category, the eddies must have a center surface salinity above 34.0 and in the second category, below 34.0.

In [ ]:
# For the sampling we again have to specify some parameters, defining when, where and which 
# eddies to sample.
# Because the dataset containing the sampled eddies can grow huge for larger regions and/or
# longer time periods, `eddytools.sample.sample()` writes the samples for each individual
# eddy to individual netcdf-files on disk!
sample_parameters = {'model': 'ORCA',
                     'grid': 'latlon',
                     'start_time': '1976-01-03', # time range start
                     'end_time': '1976-12-31', # time range end
                     'calendar': 'standard', # calendar, must be either 360_day or standard
                     'max_time': 73, # maximum length of tracks to consider
                                     # (model time steps)
                     'lon1': 172, # minimum longitude of detection region
                     'lon2': -178, # maximum longitude
                     'lat1': -58, # minimum latitude
                     'lat2': -52, # maximum latitude
                     'type': 'anticyclonic', # type of eddy
                     'lifetime': 5, # length of the eddy's track in days
                     'size': 25, # eddy size (diameter in km)
                     'd_surr': 1, # surroundings size
                     'range': False, # sample eddy within a range of `var_range`
                     'ds_range': data_int.isel(z=9), # dataset of `var_range`
                     'var_range': ['votemper'], # variable to base the range on
                     'value_range': [[4, 7],], # range of `var_range`
                     'split': False, # split eddies at a threshold in below and above
                     'ds_split': data_int.isel(z=0), # dataset of `var_split`
                     'var_split': ['vosaline'], # variable to base split on
                     'value_split': [34.0,], # split eddies at this value
                     'sample_vars': ['votemper'], # variables to sample
                     'save_location': datapath, # where to store the netcdf files 
                     'save_name': 'test'}

The sampling can take quite long as for every eddy that fits the criteria we need to read data from disk at every time step  
The names of the files will be defined by `save_name`, the criteria you specify and the eddy number  
For this sampling parameters, the file name of the first eddy will be  
`test.anticyclonic.larger_25.longer_5.0000001.nc`  

If you set `'split'` to `True`, the file names will differ for the two categories.  
`test.anticyclonic.larger_25.longer_5.0000001.above_thr.nc` for eddies that are above `'value_split'`  
`test.anticyclonic.larger_25.longer_5.0000001.below_thr.nc`
for eddies that are below `'value_split'`.

In [ ]:
et.sample.sample(tracks, data_int, sample_parameters)

# AVERAGING

We can now average over the sampled eddies.  
As each eddy has its own file, we first need to find out how many files/samples there are, so we can loop over them and then store the datasets in a dictionary.  
Note that for large region, i.e. a lot of samples the resulting dataset could be too large to fit into memory. The `chunks={}` argument to `xr.open_dataset()` is an attempt to overcome this, however I do not know yet whether this has a lot of effect! One could also split the samples into several parts and then work on each part seperately (the number of eddies going into each average is stored, so one could later do a weighted average over the different parts!)

In [ ]:
# Find out number of samples
samples = {}
num_samples = len(glob(sample_parameters['save_location']
                       + sample_parameters['save_name'] + '.anticyclonic.larger_25.longer_5.*.nc'))

In [ ]:
# Now we load each stored sample
for i in np.arange(1, num_samples + 1):
    snum = "%07d" % (i,)
    samples[i] = xr.open_dataset(sample_parameters['save_location'] + sample_parameters['save_name'] +
                                 '.anticyclonic.larger_25.longer_5.' + str(snum) + '.nc', chunks={})

First a "preparation" is performed: Basically, all eddies are interpolated onto a normalized (in length) section crossing them through the eddy center. This section's orientation can be defined by the argument `section`, which can be either `'zonal'` (default) or `'meridional'`. The length of this section can be defined with argument `interp_vec`. A larger number gives a finer resolution of the interpolated data, but might not necessarily be useful (if input data is coarser resolution for example).    
For every variable specified, at every time step, the values and anomalies (with respect to surroundings) are interpolated (with method `method`) onto the normalized section and stored according to the month the eddy was first detected. The depth profile of the surroundings is stored as well.  
For available interpolation methods please have a look in the documentation of the underlying function [`scipy.interpolate.interp1d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html).

In [ ]:
normalized_eddies = et.average.prepare(samples, ['votemper'], interp_vec=41, interp_method='nearest', section='zonal')

In [ ]:
normalized_eddies['votemper_anom']['07'].shape

Here for example, 5 eddies originating in July have been stored with a maximum length of 37 time steps. 11 is the length of the depth dimension, 41 the length of the normalized section.

Next we can average these samples in different ways.  
Three possibilities are given with `average`  
1. seasonal -> bins the samples (of all available years) into four seasons (DJF, MAM, JJA, SON) and averages them into a seasonal climatology  
2. monthly  -> does the same for each month (so you end up with a monthly climatology)  
3. total    -> averages over all available eddies  
The three methods all return the means, standard deviations (across eddies), and the number of eddies that went into the derived quantities for a) the variables specified, b) their anomalies to the surroundings, c) the surroundings. These results are stored in the output dictionary under `output['ave']['mean'][period]['variable']`, `output['ave']['mean'][period]['variable_anom']`, `output['ave']['mean'][period]['variable_around']`, respectively. `period` refers to 1. `DJF`, `MAM`, `JJA` or `SON`, 2. `01`, `02`, `03`, etc., representing the month, or in case of the total average thie layer in the dictionary does not exist.  
Additionally, the averaged temporal evolution of the eddy centers will be stored under `output['evo'][...]`.

In [ ]:
season_eddies = et.average.seasonal(normalized_eddies, ['votemper'])

In [ ]:
plt.pcolormesh(season_eddies['ave']['mean']['DJF']['votemper_anom'], vmin=-1, vmax=1, cmap=cm.balance)
plt.colorbar()

In [ ]:
monthly_eddies = et.average.monthly(normalized_eddies, ['votemper'])

In [ ]:
plt.pcolormesh(monthly_eddies['ave']['mean']['03']['votemper_anom'], vmin=-1, vmax=1, cmap=cm.balance)
plt.colorbar()

In [ ]:
total_eddies = et.average.total(normalized_eddies, ['votemper'])

In [ ]:
plt.pcolormesh(total_eddies['ave']['mean']['votemper_anom'], vmin=-1, vmax=1, cmap=cm.balance)
plt.colorbar()